# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import nltk
import import_ipynb
from challenge_1 import clean_up
from challenge_1 import tokenize
from challenge_1 import stem_and_lemmatize
from challenge_1 import remove_stopwords
import pandas as pd
import numpy as np

importing Jupyter notebook from challenge_1.ipynb


In [2]:
samples_size = 20000
top_words_size = 3000

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(txt)

{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}

In [4]:
tweets = pd.read_csv('../data/Sentiment140.csv')
tweets.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
tweets.shape

(1600000, 6)

In [6]:
tweets_sample = tweets.sample(n=samples_size)
tweets_sample.shape

(20000, 6)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [7]:
# your code here
tweets_sample['text_processed'] = tweets_sample['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)
tweets_sample['category'] = tweets_sample.target == 4

In [8]:
tweets_sample['category'].value_counts()

True     10112
False     9888
Name: category, dtype: int64

In [9]:
tweets_sample.head(10)

,target,ids,date,flag,user,text,text_processed,category
662192,0,2243523920,Fri Jun 19 13:33:30 PDT 2009,NO_QUERY,kpie,Internet is finally working which means it's t...,"[internet, final, work, mean, time, updat, iph...",False
797969,0,2328362166,Thu Jun 25 09:29:22 PDT 2009,NO_QUERY,Skirmy,"Okay, me &amp; Blizge decided to go to prom......","[okay, amp, blizg, decid, go, prom, way, wish,...",False
521950,0,2192577167,Tue Jun 16 07:14:35 PDT 2009,NO_QUERY,NSalassi,@Teradawn I think the fever is back...I'm free...,"[teradawn, think, fever, back, freez, never, c...",False
523194,0,2193042696,Tue Jun 16 07:55:04 PDT 2009,NO_QUERY,goyebr,@hstuart3 oh no!! Poor Jack and mom!!,"[hstuart, oh, poor, jack, mom]",False
1303275,4,2009050254,Tue Jun 02 15:04:15 PDT 2009,NO_QUERY,cherryoranges,Half an hour and it's dance duty for me!! I ha...,"[half, hour, danc, duti, fun, chillin, bex, cl...",True
544179,0,2201023381,Tue Jun 16 19:41:55 PDT 2009,NO_QUERY,bicfox,Not looking forward to packing up my whole roo...,"[look, forward, pack, whole, room, tomorrow, l...",False
1377792,4,2051932152,Fri Jun 05 23:10:55 PDT 2009,NO_QUERY,melissatellez,on myspace. this is my page incase you wanna s...,"[myspac, thi, page, inca, wan, na, see, add]",True
1591625,4,2191597969,Tue Jun 16 05:36:15 PDT 2009,NO_QUERY,smartie1_,Chuck is on tonight. Gotta love Zachary Levi,"[chuck, tonight, got, ta, love, zachari, levi]",True
1167854,4,1980091542,Sun May 31 05:39:20 PDT 2009,NO_QUERY,PatriciaMaurie,I saw a baby at the church and he was so cute.,"[saw, babi, church, wa, cute]",True
1332710,4,2016249547,Wed Jun 03 06:26:44 PDT 2009,NO_QUERY,kikamini,Woke up to it's my dogs birthday. Now it's stu...,"[woke, dog, birthday, stuck, head, thank, tim]",True


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [11]:
# your code here# your code here
from nltk import FreqDist
import random

In [12]:
words=[]
for text in tweets_sample.text_processed:
    for el in text:
        words.append(el)
        
all_words = FreqDist(words)

In [13]:
top_words = all_words.most_common(top_words_size)

In [14]:
top_words_w = dict(top_words).keys()

In [15]:
print('first 10 words from the top words')
list(top_words_w)[:10]

first 10 words from the top_5000 words


['go', 'get', 'day', 'wa', 'thi', 'good', 'work', 'like', 'love', 'quot']

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [16]:
documents = list(zip(tweets_sample.text_processed, tweets_sample.category))
random.shuffle(documents)

In [40]:
documents[1]

(['thi',
  'bad',
  'drive',
  'condit',
  'realli',
  'raini',
  'spray',
  'mean',
  'hard',
  'see',
  'servic',
  'x'],
 False)

In [17]:
def find_features(document):
    word = set(document)
    features = {}
    for w in top_words_w:
        features[w] = (w in word)
    return features

In [18]:
feature_set = [(find_features(tweet), cat) for (tweet, cat) in documents]

In [41]:
feature_set[:1]

[({'go': False,
   'get': False,
   'day': False,
   'wa': False,
   'thi': False,
   'good': False,
   'work': False,
   'like': True,
   'love': False,
   'quot': False,
   'u': False,
   'got': False,
   'today': False,
   'time': True,
   'want': False,
   'lol': False,
   'back': False,
   'thank': False,
   'one': False,
   'know': True,
   'miss': False,
   'see': False,
   'im': False,
   'think': False,
   'feel': False,
   'amp': False,
   'realli': False,
   'night': False,
   'watch': False,
   'need': False,
   'hope': False,
   'still': False,
   'home': False,
   'well': False,
   'make': False,
   'new': False,
   'na': False,
   'oh': False,
   'look': False,
   'last': False,
   'ha': False,
   'much': False,
   'morn': False,
   'twitter': False,
   'come': False,
   'wish': False,
   'tomorrow': False,
   'haha': False,
   'great': False,
   'wait': False,
   'sleep': False,
   'fun': False,
   'happi': False,
   'tri': False,
   'sad': False,
   'take': False,
   '

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [26]:
train_set, test_set = feature_set[:int(len(tweets_sample)*0.8)], feature_set[int(len(tweets_sample)*0.8):]
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(100)

Most Informative Features
                    sore = True            False : True   =     26.2 : 1.0
                 depress = True            False : True   =     20.0 : 1.0
            followfriday = True             True : False  =     18.6 : 1.0
                     ach = True            False : True   =     16.6 : 1.0
                 headach = True            False : True   =     15.9 : 1.0
                     sad = True            False : True   =     15.8 : 1.0
               goodnight = True             True : False  =     13.9 : 1.0
                     yum = True             True : False  =     11.4 : 1.0
                  suffer = True            False : True   =     11.2 : 1.0
                   burnt = True            False : True   =     11.2 : 1.0
                   delay = True            False : True   =     10.5 : 1.0
                 congrat = True             True : False  =     10.2 : 1.0
                     vip = True             True : False  =     10.1 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [20]:
nltk.classify.accuracy(classifier, test_set)

0.73325

In [35]:
tweet_text = 'best thing to ever happen to me'
tweet_predict = remove_stopwords(stem_and_lemmatize(tokenize(clean_up(tweet_text))))
feature_set_pred = find_features(tweet_predict)
classifier.classify(feature_set_pred)

True

In [38]:
tweet_text = 'great, wtf is this shit'
tweet_predict = remove_stopwords(stem_and_lemmatize(tokenize(clean_up(tweet_text))))
feature_set_pred = find_features(tweet_predict)
classifier.classify(feature_set_pred)

False

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [23]:
# your code here
classifier.show_most_informative_features(100)

Most Informative Features
                    sore = True            False : True   =     26.2 : 1.0
                 depress = True            False : True   =     20.0 : 1.0
            followfriday = True             True : False  =     18.6 : 1.0
                     ach = True            False : True   =     16.6 : 1.0
                 headach = True            False : True   =     15.9 : 1.0
                     sad = True            False : True   =     15.8 : 1.0
               goodnight = True             True : False  =     13.9 : 1.0
                     yum = True             True : False  =     11.4 : 1.0
                  suffer = True            False : True   =     11.2 : 1.0
                   burnt = True            False : True   =     11.2 : 1.0
                   delay = True            False : True   =     10.5 : 1.0
                 congrat = True             True : False  =     10.2 : 1.0
                     vip = True             True : False  =     10.1 : 1.0

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [24]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [25]:
# your code here